This notebook generates maps for the UI mockup answering the following 3 questions:
* Q1: What samples are near certain types of facilities (e.g. water suppy systems, waste disposal sites, etc.)?

# Setup


In [2]:
%%capture
!pip install mapclassify --upgrade --quiet
!pip install SPARQLWrapper --upgrade --quiet
!pip install rdflib

In [3]:
#from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For map layer control
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper2, JSON, GET, POST, DIGEST   # For querying SPARQL endpoints
import rdflib                                                      # For working with URIs

def convertToDataframe(results):
  d = []
  for x in results.bindings:
        row = {}
        for k in x:
            v = x[k]
            vv = rdflib.term.Literal(v.value, datatype=v.datatype).toPython()  # type: ignore[no-untyped-call]
            row[k] = vv
        d.append(row)
  df = pd.DataFrame(d)
  return df

def convertS2ListToQueryString(s2):
  s2list = s2['s2cell'].tolist()
  s2list = [s2.replace("http://stko-kwg.geog.ucsb.edu/lod/resource/","kwgr:") for s2 in s2list]
  s2_values_string = " ".join(s2list)
  return s2_values_string


In [4]:
#for interactive widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

# Q1 - What samples in Maine are near a certain types of facilities (by NAICS Industry or Group Code)?


In [5]:
industry = "221320 (Sewage Treatment)" # @param ["5622 (Waste Treatment and Disposal)","3222 (Converted Paper Manufacturing)", "221310 (Water Supply and Irrigation)", "221320 (Sewage Treatment)","3261 (Plastics Product Manufacturing)","3133 (Textile and Fabric Finishing and Coating)","3251 (Basic Chemical Manufacturing)","3255 (Paint, Coating, and Adhesive Manufacturing)", "3364 (Aerospace Product and Parts)","812320 (Drycleaning and Laundry Services)", "561740 (Carpet and Upholstery Cleaning Services)"]{"allow-input":true}

icode = industry.split()[0]
print(icode)

221320


## Queries

In [17]:
server = 'https://frink.apps.renci.org/' # @param ['https://frink.apps.renci.org/', 'https://gdb.acg.maine.edu:7201/repositories/']

### Facilities and their location

In [6]:
#WRetrieve facility details
q2 = '''
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX naics: <http://w3id.org/fio/v1/naics#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX fio: <http://w3id.org/fio/v1/fio#>

#Where are landfills or dod facilities

select DISTINCT ?facility ?facWKT ?facilityName ?industry ?industryName where {
    #SERVICE <repository:Spatial>{
    #    #determine near distance
    #    ?s2 kwg-ont:sfTouches|owl:sameAs ?s2neighbor.
    #    ?s2neighbor rdf:type kwg-ont:S2Cell_Level13.
    #}
    #SERVICE <repository:FIO>{
        #find facilities
        #?s2neighbor kwg-ont:sfContains ?facility.
            ?facility fio:ofIndustry ?industryGroup;
            fio:ofIndustry ?industryCode ;
        		geo:hasGeometry/geo:asWKT ?facWKT;
        		rdfs:label ?facilityName.
        ?industryCode a naics:NAICS-IndustryCode;  # only NAICS specific industry codes
            fio:subcodeOf ?industryGroup ;
            rdfs:label ?industryName.
        '''
if len(icode)>4:
  q2 += 'VALUES ?industryCode {naics:NAICS-' + str(icode) + '}.'
else:
  q2 += 'VALUES ?industryGroup {naics:NAICS-' + str(icode) + '}.'

q2 += '''
    #}

}
'''

#WRetrieve S2 cells
q3 = '''
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX naics: <http://w3id.org/fio/v1/naics#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX fio: <http://w3id.org/fio/v1/fio#>

#Where are landfills or dod facilities

select DISTINCT ?s2cell where {
        #find facilities
        ?s2cell rdf:type kwg-ont:S2Cell_Level13 ;
                kwg-ont:sfContains ?facility.
            ?facility fio:ofIndustry ?industryGroup;
            fio:ofIndustry ?industryCode ;
        		geo:hasGeometry/geo:asWKT ?facWKT;
        		rdfs:label ?facilityName.
        ?industryCode a naics:NAICS-IndustryCode;  # only NAICS specific industry codes
            fio:subcodeOf ?industryGroup ;
            rdfs:label ?industryName.
        '''
if len(icode)>4:
  q3 += 'VALUES ?industryCode {naics:NAICS-' + str(icode) + '}.'
else:
  q3 += 'VALUES ?industryGroup {naics:NAICS-' + str(icode) + '}.'

q3 += '''
} GROUP BY ?s2cell
'''

#print(q1)
print(q2)
print(q3)


PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX naics: <http://w3id.org/fio/v1/naics#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX fio: <http://w3id.org/fio/v1/fio#>

#Where are landfills or dod facilities

select DISTINCT ?facility ?facWKT ?facilityName ?industry ?industryName where {
    #SERVICE <repository:Spatial>{
    #    #determine near distance
    #    ?s2 kwg-ont:sfTouches|owl:sameAs ?s2neighbor.
    #    ?s2neighbor rdf:type kwg-ont:S2Cell_Level13.
    #}
    #SERVICE <repository:FIO>{
        #find facilities
        #?s2neighbor kwg-ont:sfContains ?facility.
            ?facility fio:ofIndustry ?industryGroup;
            fio:ofIndustry ?industryCode ;


In [7]:
endpointGET = f'{server}/fiokg/sparql'


sparqlGET = SPARQLWrapper2(endpointGET)
sparqlGET.setHTTPAuth(DIGEST)
#sparqlGET.setCredentials('sawgraph-endpoint', 'skailab')
sparqlGET.setMethod(GET)
sparqlGET.setReturnFormat(JSON)

sparqlGET.setQuery(q2)
facility_result = sparqlGET.query()
facilities = convertToDataframe(facility_result)
#print(facilities.info())
facilities

,facility,facWKT,facilityName,industryName
0,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-88.05284 30.66353),ARMSTRONG WORLD INDUSTRIES INC,Sewage Treatment Facilities
1,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-114.707222 32.727222),US DOI BUREAU OF RECLAMATION YUMA DESALTING PR...,Sewage Treatment Facilities
2,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-87.70049 34.826496),CYPRESS CREEK WATER TREATMENT PLANT,Sewage Treatment Facilities
3,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-111.359778 34.286056),AMERICAN GULCH WASTEWATER TREATMENT PLANT,Sewage Treatment Facilities
4,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-112.10627 33.42424),23RD AVENUE WASTEWATER TREATMENT PLANT,Sewage Treatment Facilities
...,...,...,...,...
1919,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-93.889288 36.435809),JOHN BRADLEY MEARA,Sewage Treatment Facilities
1920,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-93.66818 34.08147),JARED MYRKLE,Sewage Treatment Facilities
1921,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-94.2036 36.3895),CITY OF BENTONVILLE - BENTONVILLE WATER RESOUR...,Sewage Treatment Facilities
1922,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-94.0861 36.14333),MICHAEL COLEMAN HOME,Sewage Treatment Facilities


In [8]:
sparqlGET.setQuery(q3)
s2_result = sparqlGET.query()
s2 = convertToDataframe(s2_result)
#print(facilities.info())
s2

,s2cell
0,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
1,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
2,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
3,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
4,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
...,...
225,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
226,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
227,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
228,http://stko-kwg.geog.ucsb.edu/lod/resource/s2....


### Neighborhood around facilities

In [9]:
s2list = s2['s2cell'].tolist()
s2list = [s2.replace("http://stko-kwg.geog.ucsb.edu/lod/resource/","kwgr:") for s2 in s2list]
s2_values_string = " ".join(s2list)
#print(s2_values_string)

q4 = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>

SELECT ?s2cell WHERE {
	?s2neighbor spatial:connectedTo kwgr:administrativeRegion.USA.23 .
  VALUES ?s2neighbor {''' + s2_values_string + '''}
  ?s2neighbor kwg-ont:sfTouches | owl:sameAs ?s2cell.
 }'''


sparqlGET = SPARQLWrapper2("https://frink.apps.renci.org/spatialkg/sparql")
sparqlGET.setHTTPAuth(DIGEST)
sparqlGET.setMethod(POST)
sparqlGET.setReturnFormat(JSON)

sparqlGET.setQuery(q4)
s2_filtered_result = sparqlGET.query()
s2_filtered = convertToDataframe(s2_filtered_result)
s2_filtered

s2filteredlist = s2_filtered['s2cell'].tolist()
s2filteredlist = [s2.replace("http://stko-kwg.geog.ucsb.edu/lod/resource/","kwgr:") for s2 in s2filteredlist]
s2_filtered_values_string = " ".join(s2filteredlist)
print(s2_filtered_values_string)

kwgr:s2.level13.5529512162986295296 kwgr:s2.level13.5529512197346033664 kwgr:s2.level13.5529512231705772032 kwgr:s2.level13.5529512300425248768 kwgr:s2.level13.5529512403504463872 kwgr:s2.level13.5529515014844579840 kwgr:s2.level13.5529515049204318208 kwgr:s2.level13.5529515083564056576 kwgr:s2.level13.5529512266065510400 kwgr:s2.level13.5526401850749747200 kwgr:s2.level13.5526401885109485568 kwgr:s2.level13.5526401919469223936 kwgr:s2.level13.5526401988188700672 kwgr:s2.level13.5526402022548439040 kwgr:s2.level13.5526407863703961600 kwgr:s2.level13.5526407898063699968 kwgr:s2.level13.5526408001142915072 kwgr:s2.level13.5526401953828962304 kwgr:s2.level13.5526556503932141568 kwgr:s2.level13.5526556538291879936 kwgr:s2.level13.5526556641371095040 kwgr:s2.level13.5526556675730833408 kwgr:s2.level13.5526556744450310144 kwgr:s2.level13.5526556778810048512 kwgr:s2.level13.5526557122407432192 kwgr:s2.level13.5526557156767170560 kwgr:s2.level13.5526556710090571776 kwgr:s2.level13.552655152177

### Samples taken in the neighborhood

In [10]:
# Query just the PFAS graph for samples that match the S2 cells returned from the FIO graph

q5='''
# Get the number of results and all substances that have been detected at over 4.0 ng/L at samplepoints from a specific set of S2 cells (here cells with plastics manufacturers) together with the sample material type where these substances have been detected in
# This does not require access to the Spatial graph

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX qudt: <http://qudt.org/schema/qudt/>

SELECT (COUNT(DISTINCT ?subVal) as ?resultCount) (MAX(?result_value) as ?max) (GROUP_CONCAT(DISTINCT ?substance; separator=";<br> ") as ?substances) (GROUP_CONCAT(DISTINCT ?matTypeLabel; separator="; <br>") as ?materials) ?sp ?spWKT  WHERE {
      ?sp rdf:type coso:SamplePoint;
      spatial:connectedTo ?s2cell ;
  	  geo:hasGeometry/geo:asWKT ?spWKT.
    VALUES ?s2cell {''' + s2_filtered_values_string + '''}
  ?observation rdf:type coso:ContaminantObservation;
    	coso:observedAtSamplePoint ?sp;
    	coso:ofSubstance ?substance1 ;
        coso:analyzedSample ?sample ;
    	coso:hasResult ?result .
    ?sample rdfs:label ?sampleLabel;
  		coso:sampleOfMaterialType ?matType.
  ?matType rdfs:label ?matTypeLabel.
   ?result coso:measurementValue ?result_value;
   		coso:measurementUnit ?unit .
  FILTER (?result_value > 4).
  #VALUES ?unit {<http://qudt.org/vocab/unit/NanoGM-PER-L>}.
  ?unit qudt:symbol ?unit_sym.
    BIND((CONCAT(str(?result_value) , " ", ?unit_sym)) as ?subVal)
    BIND (STRAFTER(str(?substance1), "#parameter.") as ?substance)
} GROUP BY ?sp ?spWKT
'''
print(q5)

sparqlGET2 = SPARQLWrapper2(f"{server}sawgraph/sparql")
sparqlGET2.setHTTPAuth(DIGEST)
sparqlGET2.setMethod(POST)
sparqlGET2.setReturnFormat(JSON)

sparqlGET2.setQuery(q5)
samplepoint_result = sparqlGET2.query()
samplepoints = convertToDataframe(samplepoint_result)
samplepoints


# Get the number of results and all substances that have been detected at over 4.0 ng/L at samplepoints from a specific set of S2 cells (here cells with plastics manufacturers) together with the sample material type where these substances have been detected in
# This does not require access to the Spatial graph

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX qudt: <http://qudt.org/schema/qudt/>

SELECT (COUNT(DISTINCT ?subVal) as ?resultCount) (MAX(?result_value) as ?max) (GROUP_CONCAT(DISTINCT ?substance; separator=";<br> ") as ?substances) (GROUP_CONCAT(DISTINCT ?matTypeLabel

,resultCount,max,substances,materials,sp,spWKT
0,18,85.12,PFOS_A;<br> PFPEA_A;<br> PFPES_A;<br> PFHXA_A;...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-67.8894084 46.9393394)
1,30,53040.8,PFOS_A;<br> PFUNDA_A;<br> N-MeFOSAA_A;<br> PFP...,GROUNDWATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-68.8332705 44.8058271)
2,1,6.046,SUM_OF_5_PFAS;<br> SUM_OF_6_PFAS,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-69.4052156 44.6013181)
3,10,329.58,PFHXS;<br> PFHPA;<br> PFPEA_A;<br> PFBA_A;<br>...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-67.9056908 46.9457126)
4,80,45.9,PFPEA_A;<br> 6-2_FTS_A;<br> PFHXA_A;<br> PFOA_...,WASTE WATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-70.2415401 43.7155742)
...,...,...,...,...,...,...
891,53,18.3,PFOS_A;<br> PFPEA_A;<br> PFHXA_A;<br> PFBA_A;<...,WASTE WATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-67.8258454 46.1477254)
892,4,11.671,PFOA_A;<br> SUM_PFOA_PFOS;<br> SUM_OF_5_PFAS;<...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-70.5217588 44.2019848)
893,2,5.81,PFOS_A;<br> SUM_PFOA_PFOS,SLUDGE,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-70.6069654 43.1818241)
894,3,6.565,PFOS_A;<br> SUM_PFOA_PFOS,SLUDGE,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-69.9461213 43.9154139)


## Prep data for mapping

In [11]:
samplepoints['spWKT'] = samplepoints['spWKT'].apply(wkt.loads)
samplepoints = gpd.GeoDataFrame(samplepoints, geometry='spWKT')

samplepoints.set_crs(epsg=4326, inplace=True, allow_override=True)


,resultCount,max,substances,materials,sp,spWKT
0,18,85.12,PFOS_A;<br> PFPEA_A;<br> PFPES_A;<br> PFHXA_A;...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-67.88941 46.93934)
1,30,53040.8,PFOS_A;<br> PFUNDA_A;<br> N-MeFOSAA_A;<br> PFP...,GROUNDWATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-68.83327 44.80583)
2,1,6.046,SUM_OF_5_PFAS;<br> SUM_OF_6_PFAS,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-69.40522 44.60132)
3,10,329.58,PFHXS;<br> PFHPA;<br> PFPEA_A;<br> PFBA_A;<br>...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-67.90569 46.94571)
4,80,45.9,PFPEA_A;<br> 6-2_FTS_A;<br> PFHXA_A;<br> PFOA_...,WASTE WATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-70.24154 43.71557)
...,...,...,...,...,...,...
891,53,18.3,PFOS_A;<br> PFPEA_A;<br> PFHXA_A;<br> PFBA_A;<...,WASTE WATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-67.82585 46.14773)
892,4,11.671,PFOA_A;<br> SUM_PFOA_PFOS;<br> SUM_OF_5_PFAS;<...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-70.52176 44.20198)
893,2,5.81,PFOS_A;<br> SUM_PFOA_PFOS,SLUDGE,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-70.60697 43.18182)
894,3,6.565,PFOS_A;<br> SUM_PFOA_PFOS,SLUDGE,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-69.94612 43.91541)


In [12]:
facilities['facWKT'] = facilities['facWKT'].apply(wkt.loads)
facilities = gpd.GeoDataFrame(facilities, geometry='facWKT')

facilities.set_crs(epsg=4326, inplace=True, allow_override=True)

,facility,facWKT,facilityName,industryName
0,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-88.05284 30.66353),ARMSTRONG WORLD INDUSTRIES INC,Sewage Treatment Facilities
1,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-114.70722 32.72722),US DOI BUREAU OF RECLAMATION YUMA DESALTING PR...,Sewage Treatment Facilities
2,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-87.70049 34.8265),CYPRESS CREEK WATER TREATMENT PLANT,Sewage Treatment Facilities
3,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-111.35978 34.28606),AMERICAN GULCH WASTEWATER TREATMENT PLANT,Sewage Treatment Facilities
4,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-112.10627 33.42424),23RD AVENUE WASTEWATER TREATMENT PLANT,Sewage Treatment Facilities
...,...,...,...,...
1919,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-93.88929 36.43581),JOHN BRADLEY MEARA,Sewage Treatment Facilities
1920,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-93.66818 34.08147),JARED MYRKLE,Sewage Treatment Facilities
1921,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-94.2036 36.3895),CITY OF BENTONVILLE - BENTONVILLE WATER RESOUR...,Sewage Treatment Facilities
1922,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-94.0861 36.14333),MICHAEL COLEMAN HOME,Sewage Treatment Facilities


In [13]:
from branca.element import Figure

map = samplepoints.explore(name=f'<span style="color:DarkOrange;">Samples</span>', color='DarkOrange',
                           style_kwds=dict(style_function=lambda x: {"radius": float(x['properties']["max"])/8 if float(x['properties']["max"]) < 160 else 25,
                                                                     "opacity":0.3,
                                                                     "color":'DimGray',
                                                                     }),
                           marker_kwds=dict(radius=6),
                           marker_type='circle_marker',
                           popup = ["substances", "materials", "max", "resultCount"],
                                ) #
#map

samplepoints.set_crs(epsg=4326, inplace=True, allow_override=True)

,resultCount,max,substances,materials,sp,spWKT
0,18,85.12,PFOS_A;<br> PFPEA_A;<br> PFPES_A;<br> PFHXA_A;...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-67.88941 46.93934)
1,30,53040.8,PFOS_A;<br> PFUNDA_A;<br> N-MeFOSAA_A;<br> PFP...,GROUNDWATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-68.83327 44.80583)
2,1,6.046,SUM_OF_5_PFAS;<br> SUM_OF_6_PFAS,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-69.40522 44.60132)
3,10,329.58,PFHXS;<br> PFHPA;<br> PFPEA_A;<br> PFBA_A;<br>...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-67.90569 46.94571)
4,80,45.9,PFPEA_A;<br> 6-2_FTS_A;<br> PFHXA_A;<br> PFOA_...,WASTE WATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-70.24154 43.71557)
...,...,...,...,...,...,...
891,53,18.3,PFOS_A;<br> PFPEA_A;<br> PFHXA_A;<br> PFBA_A;<...,WASTE WATER,http://w3id.org/sawgraph/v1/me-egad-data#sampl...,POINT (-67.82585 46.14773)
892,4,11.671,PFOA_A;<br> SUM_PFOA_PFOS;<br> SUM_OF_5_PFAS;<...,GROUNDWATER,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-70.52176 44.20198)
893,2,5.81,PFOS_A;<br> SUM_PFOA_PFOS,SLUDGE,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-70.60697 43.18182)
894,3,6.565,PFOS_A;<br> SUM_PFOA_PFOS,SLUDGE,http://sawgraph.spatialai.org/v1/me-egad-data#...,POINT (-69.94612 43.91541)


In [14]:
#map = folium.Map()
c = 0
colors = ['MidnightBlue','Blue','DodgerBlue','DeepSkyBlue','CadetBlue','DarkCyan','LightSeaGreen','MediumSageGreen','SpringGreen','PaleVioletRed','Purple','Fuchsia','HotPink','LightPink']
for industry in list(facilities.industryName.unique()):
  #print(industry)
  #print(facilities[facilities['industryName']== industry].info())
  facilities[facilities['industryName']== industry].explore(m=map,
                                                            name=f'<span style="color:{colors[c]};">{industry}</span>',
                                                            color=colors[c],
                                                            marker_kwds=dict(radius=3),
                                                            popup=True)
  c += 1

facilities.set_crs(epsg=4326, inplace=True, allow_override=True)

,facility,facWKT,facilityName,industryName
0,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-88.05284 30.66353),ARMSTRONG WORLD INDUSTRIES INC,Sewage Treatment Facilities
1,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-114.70722 32.72722),US DOI BUREAU OF RECLAMATION YUMA DESALTING PR...,Sewage Treatment Facilities
2,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-87.70049 34.8265),CYPRESS CREEK WATER TREATMENT PLANT,Sewage Treatment Facilities
3,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-111.35978 34.28606),AMERICAN GULCH WASTEWATER TREATMENT PLANT,Sewage Treatment Facilities
4,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-112.10627 33.42424),23RD AVENUE WASTEWATER TREATMENT PLANT,Sewage Treatment Facilities
...,...,...,...,...
1919,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-93.88929 36.43581),JOHN BRADLEY MEARA,Sewage Treatment Facilities
1920,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-93.66818 34.08147),JARED MYRKLE,Sewage Treatment Facilities
1921,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-94.2036 36.3895),CITY OF BENTONVILLE - BENTONVILLE WATER RESOUR...,Sewage Treatment Facilities
1922,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-94.0861 36.14333),MICHAEL COLEMAN HOME,Sewage Treatment Facilities


## Map

In [15]:
folium.LayerControl(collapsed=False).add_to(map)
fig = Figure(width='100%', height=900)
fig.add_child(map)


In [16]:
from datetime import date

today = date.today()
fig.save(f'SAWGraph-demo_{today}_SamplesNearFacilities_{"".join(industry.split())}.html')